# Implementing Backward Propogation

In [1]:
import tensorflow as tf

# Generate Synthetic Data

### Weights and Bias

In [2]:
weights = tf.constant([2,-3.4],dtype=tf.float32)
bias = tf.constant([4.2],dtype=tf.float32)

### Generate 10,000 Samples from random distribution

In [3]:
N=1000
dims = weights.shape[0]
batch_size=16

In [4]:
samples = tf.random.normal((N,dims))

In [5]:
y = tf.tensordot(samples,weights,1)+bias

## Add Noise

In [6]:
y = y+tf.random.normal((N,))

In [7]:
y.shape

TensorShape([1000])

## Initiate random weights

In [8]:
w = tf.random.normal((dims,))
b = tf.random.normal((1,))

In [9]:
w,b = [tf.Variable(i) for i in [w,b]]

### Custom Gradient implementation for BP (Linear Function)

In [10]:
@tf.custom_gradient
def forward_pass(x,w,b):
    def grad(dy):
        #print(tf.tensordot(dy,x,1).shape)
        return None,tf.tensordot(dy,x,1), tf.reduce_mean(dy,keepdims=True)
    return tf.tensordot(x,w,1)+b, grad

In [11]:
def batch_output(n=16):
    for i in range(N//n):
        yield samples[i*n:i*n+n], i*n

### Custom Gradient implementation for BP (Error Loss)

In [12]:
@tf.custom_gradient
def loss(y,y_act):
    loss_vector = tf.math.square(y-y_act)
    def grad(dy):
        return dy*(y-y_act),dy*(y-y_act)
    return tf.reduce_mean(loss_vector), grad

In [13]:
def correct_gradients(w_grad, b_grad,alpha=0.1):
    return w-alpha*w_grad, b-alpha*b_grad

In [14]:
for i in range(5):
    for batch, batch_index in batch_output(n=batch_size):
        with tf.GradientTape() as t_graph:
            y_forward = forward_pass(batch,w,b)
            y_loss = loss(y_forward,y[batch_index:batch_index+batch_size])
        w_grad, b_grad = t_graph.gradient(y_loss,[w,b])
        w.assign_sub(0.01*w_grad), b.assign_sub(0.01*b_grad)
    print("Loss at epoch {} is {}".format(i,y_loss))

Loss at epoch 0 is 11.868980407714844
Loss at epoch 1 is 5.209050178527832
Loss at epoch 2 is 2.9803500175476074
Loss at epoch 3 is 2.169306755065918
Loss at epoch 4 is 1.8444125652313232


### We determine that weights are near equal to original Weights are near equal to original weights

In [15]:
w,b

(<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([ 2.039919 , -3.4037657], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([3.990725], dtype=float32)>)